<a target="_blank" href="https://colab.research.google.com/github/victorlymarev/pandas/blob/main/notebooks/22-categoorical.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Категориальный тип данных
#### [Ссылка на видео](https://youtu.be/I6EvevDF6C0)

In [ ]:
import pandas as pd
import os

In [ ]:
path_empl = '../tables/employees.parquet' if os.path.exists('../tables/employees.parquet') else 'https://drive.google.com/uc?id=1AARD5-eVlCxoApt5CYZebrC3Cqw42lvj'

empl = pd.read_parquet(path_empl)
empl.head()

#### Категориальный тип позволяет экономить память

In [ ]:
empl['pos_name'].memory_usage()

In [ ]:
# приводим обычную колонку к категориальному типу
empl['pos_name'] = pd.Categorical(empl['pos_name'])

empl['pos_name'].memory_usage()

In [ ]:
469952 / 264720

Экономия достигается за счет того, что в каждой ячейке хранится не строка а число

In [ ]:
empl['pos_name'].cat.categories

In [ ]:
empl['pos_name'].cat.codes

#### Категории можно сортировать

In [ ]:
# Всписке категорий указываем значения в порядке возрастания
cats = ['Уборщик',
    'Кассир',
    'Продавец-консультант',
    'Старший кассир',
    'Мастер по ремонту одежды',
    'Товаровед',
    'Бухгалтер',
    'Заместитель директора магазина',
    'Директор магазина'
]

In [ ]:
empl['pos_name'] = empl['pos_name'].cat.reorder_categories(cats, ordered=True)

In [ ]:
# empl['pos_name'] = pd.Categorical(empl['pos_name'], categories=cats, ordered=True)

In [ ]:
empl['pos_name'].head()

#### Теперь мы можем сравнивать категории

In [ ]:
empl['pos_name'] > 'Товаровед'

In [ ]:
empl['pos_name'].max()

#### Удаление порядка из категорий

In [ ]:
empl['pos_name'].cat.as_unordered()

#### Добавление новой категории

В категориальный тип нельзя добавлять новые значения, которые не находятся в списке категорий

In [ ]:
empl['pos_name'].iloc[3] = 'Мерчендайзер'

In [ ]:
empl['pos_name'] = empl['pos_name'].cat.add_categories('Мерчендайзер')
empl['pos_name'].head()

In [ ]:
empl['pos_name'].iloc[3] = 'Мерчендайзер'
empl['pos_name'].head()

Можно автоматически удалять ненужные категории

In [ ]:
(empl['pos_name']
    .cat.add_categories('Грузчик') # добавляем категорию
    .cat.remove_unused_categories() # удаляем категрию
    .head()
)

## сut

#### Функция делит непрерывную величину на интервалы

Можно указать количество равных по длинне интервалов, на которое поделится величина

In [ ]:
pd.cut(empl['salary'], bins=10)

Можно передать эти интервалы

In [ ]:
pd.cut(empl['salary'], bins=[0, 20000, 50000, 100000, 1000000])

Можно указать названия интервалов

In [ ]:
pd.cut(empl['salary'],
       bins=[0, 20000, 50000, 100000, 1000000],
       labels=['Низкая зарплата', 'Средняя зарплата',
                'Выше среднего', 'Высокая зарплата'])

Затем на основе этого можно построить график.

Чтобы категории шли в порядке  категорий необходимо в параметр sort метода value_counts передать значение False  

In [ ]:
(pd.cut(empl['salary'],
           bins=[0, 20000, 50000, 100000, 1000000],
           labels=['Низкая зарплата', 'Средняя зарплата',
                    'Выше среднего', 'Высокая зарплата'])
    .value_counts(sort=False)
    .plot(kind='bar', figsize=(8, 3))
)

## qcut

#### Делит непрерывную величину на интервалы по квантилям (так, чтобы в каждой группе было примерно равно число наблюдений)

In [ ]:
# в каждый полуинтервал попадает по 25% (100 / q = 4)
pd.qcut(empl['salary'], q=4)

In [ ]:
pd.qcut(empl['salary'], q=[0, 0.1, 0.4, 0.75, 1])

#### При помощи данного метода можно стратифицированно делить людей на группы

In [ ]:
(empl
    .query("report_dt == '2022-12-31'")
    .assign(salary_bin = lambda df: pd.qcut(df['salary'], q=5)) # делим зарплату на 5 интервалов
    .groupby(['sex', 'education', 'salary_bin']) # группируем по  полу, образованию и зарплатному интервалу
    .sample(frac=0.3, random_state=10) # берем 30% строк из каждой группы
    
)

In [ ]:
(empl
    .query("report_dt == '2022-12-31'")
    .groupby('sex') # группируем по полу
    .apply(lambda x: x.assign(salary_bin = 
                              # внутри каждой группы по полу делим зарплату на 5 интервалов
                              lambda df: pd.qcut(df['salary'], q=5))
                                            .groupby('salary_bin') # группируем по этим интервалам
                                            .sample(frac=0.3, random_state=10) # берем 30% строк из каждой группы
          )
    .reset_index(drop=True)
)

# Задания

#### Описание таблиц лежит [здесь](https://github.com/victorlymarev/pandas/tree/main/tables#%D0%BE%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D0%B5-%D1%82%D0%B0%D0%B1%D0%BB%D0%B8%D1%86)

Некоторые таблицы занимают много памяти, поэтому каждые 5-10 заданий лучше перезапускайте ноутбук.

В формулировке некоторых заданий может содержаться вариативность. Если у вас есть сомнения, что требуется в задании, попробуйте решить несколько вариантов. Если вы не понимаете задание, можете написать об этом в комментариях под видео.

В большинстве случаев внешний вид итоговой таблицы не определен. Выведите на экран наиболее адекватный вариант. То есть таблицу, из которой можно будет сделать выводы.

Курс пока находится в разработке. Вы можете помочь другим людям добавив свое решение [сюда](https://docs.google.com/forms/d/1HYTBz_KfssY3Jps2dC3n0YnEqa6WBb5OIhLo1d32Xzw/edit).

Посмотреть решения других людей можно [здесь](https://docs.google.com/spreadsheets/d/1pMDYO-9UneLbPPnEuQ2shig0TOJdQTU-zipifyAnZMk/edit?resourcekey#gid=1998648012)

### Задание 1

Разделите объем торговли на 10 равных интервалов

In [ ]:
# таблица ltc - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_ltc_sample на path_ltc_full внутри функции read_parquet

import os
import pandas as pd

# path_ltc_full = '../tables/ltc.parquet' if os.path.exists('../tables/ltc.parquet') else "https://drive.usercontent.google.com/download?id=1ZkAmVZverOV3aGwmEQGAFXgXnQ6pPsZw&export=download&authuser=1&confirm=t&uuid=b827b3e2-7c5d-4979-9d25-f1c34954ac9f&at=APZUnTUs_oUnCQujGIlgn2Zkb5VG:1693327327264"
path_ltc_sample = '../tables/ltc_sample.parquet' if os.path.exists('../tables/ltc_sample.parquet') else 'https://drive.google.com/uc?id=1XaThogOOqKjJj50LvfJ9WqutjMAC5AxA'

ltc = pd.read_parquet(path_ltc_sample)
ltc.head()

In [ ]:
# напишите свой код здесь

### Задание 2

Разделите объем торговли на 10 интервалов так, чтобы в каждом интервале лежало примерно равное число наблюдений

In [ ]:
# таблица ltc - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_ltc_sample на path_ltc_full внутри функции read_parquet

import os
import pandas as pd

# path_ltc_full = '../tables/ltc.parquet' if os.path.exists('../tables/ltc.parquet') else "https://drive.usercontent.google.com/download?id=1ZkAmVZverOV3aGwmEQGAFXgXnQ6pPsZw&export=download&authuser=1&confirm=t&uuid=b827b3e2-7c5d-4979-9d25-f1c34954ac9f&at=APZUnTUs_oUnCQujGIlgn2Zkb5VG:1693327327264"
path_ltc_sample = '../tables/ltc_sample.parquet' if os.path.exists('../tables/ltc_sample.parquet') else 'https://drive.google.com/uc?id=1XaThogOOqKjJj50LvfJ9WqutjMAC5AxA'

ltc = pd.read_parquet(path_ltc_sample)
ltc.head()

In [ ]:
# напишите свой код здесь

### Задание 3

Посчитайте симметричную KL дивергенцию между распределением зарплат 31 декабря 2021 года и распределением зарплат 31 декабря 2022 года. Для этого сначала разделите исходную таблицу на две (по датам). Затем внутри каждой таблицы разделите зарплаты на интервалы от нуля до 30 000, от 30 000 до 40 000, от 40 000 до 50 000, от 50 000 до 60 000, от 60 000 до 70 000, от 70 000 до 100 000 и от 100 000  до 200 000. После чего посчитайте долю наблюдений в каждом интервале. 

В итоге у вас должно получиться 2 массива, в которых лежат доли наблюдений в каждом интервале. Назовем их массив1 и массив2.

Посчитайте следующую величину: (массив1 - массив2) * log(массив1/массив2)
И просуммируйте значения в нем.

P.S. KL - дивергенция служит мерой похожести между двумя распределениями. 

### Задание 4

Для каждого покупателя, который был в магазине в 2022 году, посчитайте сколько раз за этот год он приходил в магазин (по числу чеков), среднюю сумму его чека, и пол (если 70 и более процентов его товаров за этот год - мужские, то это мужчина, если 70 и более процентов его товаров женские, то женщина, иначе считайте, что карточка клиента семейная. То есть в поле 3 значения). Затем при помощи функции qcut разделите количество товаров в чеке на 5 интервалов, а среднюю сумму чека на 10. Теперь на на основе трех колонок разделите покупателей на 3 равные группы. Каждый покупатель может быть только в одной группе.  

In [ ]:
# таблица sales - большая, и в некоторых случаях ваш компьютер может не справиться с ее обработкой
# поэтому лучше работайте с частью этой таблицы
# но если вы хотите попробовать поработать с полной версией таблицы,
# можете заменить переменную path_sales_2022 на path_sales внутри функции read_parquet

import os
import pandas as pd

path_sales_2022 = '../tables/sales_2022.parquet' if os.path.exists('../tables/sales_2022.parquet') else 'https://drive.google.com/uc?id=17e7FwXVdsWc2aziK9s5KidIvPcfKt9F5'
# path_sales = '../tables/sales.parquet' if os.path.exists('../tables/sales.parquet') else "https://drive.usercontent.google.com/download?id=15KwSxyM6hpNABGe6_vsrFZvD09VfHFyK&export=download&authuser=1&confirm=t&uuid=115bd48c-cc2c-4f2a-8b42-be5ca6ef6db8&at=APZUnTUVb8nfNANw5wr9Cad7PJ3U:1693327774694"

sales = pd.read_parquet(path_sales_2022)
sales.head()

In [ ]:
path_goods_descr = '../tables/goods_description.parquet' if os.path.exists('../tables/goods_description.parquet') else 'https://drive.google.com/uc?id=1YbiD02Rev-X_WWV9nPSG1zZFmEh2JjPh'

goods_descr = pd.read_parquet(path_goods_descr)
goods_descr.head()

In [ ]:
# напишите свой код здесь